In [81]:
# Reference: Extracting Song Data From the Spotify API Using Python
# https://towardsdatascience.com/extracting-song-data-from-the-spotify-api-using-python-b1e79388d50

# Import dependencies
import pandas as pd
from pathlib import Path
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from config import cid, secret

In [82]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


In [84]:
# Create variable playlist_link and set to the playlist url
playlist_link = "https://open.spotify.com/playlist/2YRe7HRKNRvXdJBp9nXFza"
# Create variable playlist that only contains the uri ids
playlist_URI = playlist_link.split("/")[-1]

In [85]:
# Create the playlist tracks, get items, paginate through next results and extend all tracks
results = sp.playlist_tracks(playlist_URI)
tracks = results['items']
while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])

In [86]:
# Checking number of records in tracks
len(tracks)

650

In [87]:
# Create a list called outcome and loop through the Spotify playlist
outcome = []

#for track in sp.playlist_tracks(playlist_URI)["items"]:
for track in tracks:
  track_data = {}
  
  #URI
  track_uri = track["track"]["uri"]

  #Track name
  track_name = {'Track Name' : track["track"]["name"]}

  #Add to track_data dictionary
  track_data.update(track_name)

  #Main Artist
  artist_uri = track["track"]["artists"][0]["uri"]
  artist_info = sp.artist(artist_uri)
  
  #Name, ID, popularity, genre
  artist_name = {'Artist Name' : track["track"]["artists"][0]["name"]}
  track_data.update(artist_name)

  artist_id = {'Artist ID' : track["track"]["artists"][0]["id"]}
  track_data.update(artist_id)

  artist_pop = {'Artist Popularity' : artist_info["popularity"]}
  track_data.update(artist_pop)

  artist_genres = {'Artist genre' : artist_info["genres"]}
  track_data.update(artist_genres)

  #Album, ID
  album = {'Album Name' : track["track"]["album"]["name"]}
  track_data.update(album)

  album_id = {'Album ID' : track["track"]["album"]["id"]}
  track_data.update(album_id)

  #Popularity of the track
  track_pop = {'Track Popularity' : track["track"]["popularity"]}
  track_data.update(track_pop)

  #Audio features
  audio_features = (sp.audio_features(track_uri))[0]
  track_data.update(audio_features)

  # Add track_data in outcome list.
  outcome.append(track_data)

outcome[0:5]

[{'Track Name': 'Shape of You',
  'Artist Name': 'Ed Sheeran',
  'Artist ID': '6eUKZXaKkcviH0Ku9w2n3V',
  'Artist Popularity': 91,
  'Artist genre': ['pop', 'uk pop'],
  'Album Name': '÷ (Deluxe)',
  'Album ID': '3T4tUhGYeRNVUGevb0wThu',
  'Track Popularity': 85,
  'danceability': 0.825,
  'energy': 0.652,
  'key': 1,
  'loudness': -3.183,
  'mode': 0,
  'speechiness': 0.0802,
  'acousticness': 0.581,
  'instrumentalness': 0,
  'liveness': 0.0931,
  'valence': 0.931,
  'tempo': 95.977,
  'type': 'audio_features',
  'id': '7qiZfU4dY1lWllzX7mPBI3',
  'uri': 'spotify:track:7qiZfU4dY1lWllzX7mPBI3',
  'track_href': 'https://api.spotify.com/v1/tracks/7qiZfU4dY1lWllzX7mPBI3',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7qiZfU4dY1lWllzX7mPBI3',
  'duration_ms': 233713,
  'time_signature': 4},
 {'Track Name': 'Blinding Lights',
  'Artist Name': 'The Weeknd',
  'Artist ID': '1Xyo4u8uXC1ZmMpatF05PJ',
  'Artist Popularity': 94,
  'Artist genre': ['canadian contemporary r&b', 'cana

In [88]:
# Create a DataFrame df to save the data in dictionary.
df = pd.DataFrame(outcome)

In [89]:
# Display first 5 rows of the dataframe
df.head()

,Track Name,Artist Name,Artist ID,Artist Popularity,Artist genre,Album Name,Album ID,Track Popularity,danceability,energy,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Shape of You,Ed Sheeran,6eUKZXaKkcviH0Ku9w2n3V,91,"[pop, uk pop]",÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,85,0.825,0.652,...,0.0931,0.931,95.977,audio_features,7qiZfU4dY1lWllzX7mPBI3,spotify:track:7qiZfU4dY1lWllzX7mPBI3,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,https://api.spotify.com/v1/audio-analysis/7qiZ...,233713,4
1,Blinding Lights,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,94,"[canadian contemporary r&b, canadian pop, pop]",After Hours,4yP0hdKOZPNshxUOjY0cZj,90,0.514,0.730,...,0.0897,0.334,171.005,audio_features,0VjIjW4GlUZAMYd2vXMi3b,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4
2,Dance Monkey,Tones And I,2NjfBq1NflQcKSeiDooVjY,72,"[australian pop, pop]",Welcome To The Madhouse (Deluxe),1AZjTOqvw2ZogWZxnDlhEN,76,0.824,0.587,...,0.1490,0.514,98.029,audio_features,2N8m6CYs74qQO4mjVcXO30,spotify:track:2N8m6CYs74qQO4mjVcXO30,https://api.spotify.com/v1/tracks/2N8m6CYs74qQ...,https://api.spotify.com/v1/audio-analysis/2N8m...,209438,4
3,rockstar (feat. 21 Savage),Post Malone,246dkjvS1zLTtiykXe5h60,89,"[dfw rap, melodic rap, rap]",beerbongs & bentleys,6trNtQUgC8cgbWcqoMYkOR,84,0.585,0.520,...,0.1310,0.129,159.801,audio_features,0e7ipj03S05BNilyu5bRzt,spotify:track:0e7ipj03S05BNilyu5bRzt,https://api.spotify.com/v1/tracks/0e7ipj03S05B...,https://api.spotify.com/v1/audio-analysis/0e7i...,218147,4
4,Someone You Loved,Lewis Capaldi,4GNC7GD6oZMSxPGyXy4MNB,81,"[pop, uk pop]",Divinely Uninspired To A Hellish Extent,5658aM19fA3JVwTK6eQX70,87,0.501,0.405,...,0.1050,0.446,109.891,audio_features,7qEHsqek33rTcFNT9PFqLf,spotify:track:7qEHsqek33rTcFNT9PFqLf,https://api.spotify.com/v1/tracks/7qEHsqek33rT...,https://api.spotify.com/v1/audio-analysis/7qEH...,182161,4


In [90]:
# Convert dataframe to csv and write to a file in Resource folder.
file_path = Path('Resources\spotify_dataset_new.csv')
df.to_csv(file_path, sep=',', encoding='utf-8', index=False)